# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235183206626                   -0.50    7.0
  2   -7.250203686843       -1.82       -1.40    1.0
  3   -7.251300626016       -2.96       -2.02    4.0
  4   -7.251248420206   +   -4.28       -2.18    4.0
  5   -7.251305496680       -4.24       -2.41    3.0
  6   -7.251337764641       -4.49       -3.10    2.0
  7   -7.251338774510       -6.00       -4.07    2.0
  8   -7.251338790139       -7.81       -4.06    4.0
  9   -7.251338797065       -8.16       -4.39    3.0
 10   -7.251338798007       -9.03       -4.61    1.0
 11   -7.251338798695       -9.16       -5.76    3.0
 12   -7.251338798704      -11.05       -6.02    4.0
 13   -7.251338798704      -12.61       -6.44    3.0
 14   -7.251338798705      -13.13       -6.73    2.0
 15   -7.251338798705      -13.75       -7.22    2.0
 16   -7.251338798705   +    -Inf       -7.60    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0774023933587942
[ Info: Arnoldi iteration step 2: normres = 0.6087952348867076
[ Info: Arnoldi iteration step 3: normres = 0.7020813804934246
[ Info: Arnoldi iteration step 4: normres = 0.2496739133132066
[ Info: Arnoldi iteration step 5: normres = 0.5142172260640961
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.85e-02, 7.40e-02, 4.40e-01, 2.55e-01, 9.27e-04)
[ Info: Arnoldi iteration step 6: normres = 0.26480693109098474
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.32e-03, 1.71e-01, 1.49e-01, 9.19e-02, 9.39e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08466704989672513
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.73e-04, 1.27e-02, 9.30e-03, 3.66e-02, 6.41e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10277416339581776
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.81e-0